<a href="https://colab.research.google.com/github/codedyasai/Python_MachineLearning/blob/main/09_%ED%8A%B8%EB%A6%AC%EC%9D%98_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 앙상블 학습(Ensemble Learning)

- 여러 개의 의사결정트리를 결합하여 하나의 결정트리보다 더 좋은 성능을 내는 머신러닝 학습기법

- 앙상블 학습의 핵심은 약한 분류기를 병렬(배깅) 또는 직렬(부스팅)으로 결합하여 강력한 분류기로 만드는 것이다.

- 정형 데이터를 다루는데 가장 뛰어난 성과를 나타내는 알고리즘

- 종류:
  1. 배깅: 동일 알고리즘을 병렬로 사용
  2. 부스팅: 동일 알고리즘을 직렬로 사용
  3. 보팅: 다른 알고리즘을 병렬로 사용


## 랜덤 포레스트(Random Forest)

- 앙상블 학습의 대표 주자
- 결정트리를 랜덤하게 만들어 결정트리 숲을 만든다.
- 랜덤 포레스트는 각각의 트리를 훈련하기 위한 데이터를 랜덤하게 만든다.
- 부트스트랩 샘플: 부트스트랩 방식으로 샘플링하여 분류한 데이터
- 부트스트랩 샘플은 훈련세트의 크기와 같게 만든다.
- 랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻는다.

In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install mglearn
import mglearn

# 음수표현 라이브러리
plt.rcParams['axes.unicode_minus'] = False

# 경고무시
import warnings
warnings.filterwarnings("ignore")

# 매직명령어 : 시각화 결과가 노트북에 포함되도록
%matplotlib inline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 8.5 MB/s eta 0:00:00


In [ ]:
wine = pd.read_csv('https://raw.githubusercontent.com/rickiepark/hg-mldl/master/wine.csv')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


- RandomForestClassifier: 분류
- RandomForestResressor: 회귀
- sklearn의 랜덤 포레스트는 100개의 결정트리를 훈련하는 방식
- 분류일 때 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스로 결과 예측하고, 회귀일 때는 각 트리의 예측값을 평균하여 결과 예측
- 트리알고리즘의 가장 큰 단점은 과대적합이 발생한다는 점이다.


In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, random_state= 42)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

# n_jobs= -1: 시스템에 CPU 코어를 모두 사용한다.
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
# return_train_score= True : 검증세트 뿐만 아니라 훈련세트의 점수도 같이 반환한다
scores = cross_validate(rf, train_input, train_target, n_jobs= -1, return_train_score= True)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.997844759088341 0.8914208392565683


In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23155241 0.49706658 0.27138101]


- 부트스트랩 샘플을 이용하여 학습하는 랜덤 포레스트는 데이터의 중복을 가져올 수 있지만, 어떤 데이터들은 학습에 사용되지 않을 수도 있다.
- OOB(Out Of Bag): 모델이 학습을 할떄 사용되지 않은 데이터들
- OOB를 마치 검증세트처럼 사용하여 자체 평가를 하는 기능을 제공한다.

In [ ]:
# oob_score 매개변수에 True(기본값 False)
rf = RandomForestClassifier(oob_score= True, n_jobs= -1, random_state= 42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8981937602627258


## 하이퍼파라미터 튜닝

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': [100],
    'max_depth': [6,8,10,12],
    'min_samples_leaf': [8,12,18],
    'min_samples_split': [8,16,20]
}

model = RandomForestClassifier(n_jobs= -1, random_state= 42)
gs = GridSearchCV(model, params, n_jobs= -1, cv= 2)
gs.fit(train_input, train_target)

GridSearchCV(cv=2, estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [6, 8, 10, 12],
                         'min_samples_leaf': [8, 12, 18],
                         'min_samples_split': [8, 16, 20],
                         'n_estimators': [100]})

In [ ]:
gs.best_params_

{'max_depth': 10,
 'min_samples_leaf': 8,
 'min_samples_split': 8,
 'n_estimators': 100}

In [ ]:
gs.best_score_

0.8766420361247947

## Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
# 개별 모델 객체 생성
# L1(Lasso), L2(Ridge)
lr = LogisticRegression(solver= 'liblinear')
knn = KNeighborsClassifier(n_neighbors= 3)

# 소프트 보팅으로 구현
vo = VotingClassifier(estimators= [('LR', lr), ('KNN', knn)], voting= 'soft')

# 학습
vo.fit(train_input, train_target)

# 예측
pred = vo.predict(test_input)

from sklearn.metrics import accuracy_score
# 정확도
print('Voting 분류기 정확도: ', accuracy_score(test_target, pred))

# 모델별 학습/예측/평가
models = [lr, knn]
for model in models:
    model.fit(train_input, train_target)
    pred = model.predict(test_input)
    model_name = model.__class__.__name__
    score = accuracy_score(test_target, pred)
    print(f'{model_name} 정확도: {score:.4f}')

Voting 분류기 정확도:  0.848
LogisticRegression 정확도: 0.7655
KNeighborsClassifier 정확도: 0.8425
